# Create Custom Shape

Create a slab with a complex cutout pattern.
By default, this notebook generates a structure resembling a transistor:
- Two vertical side walls
- Two parallel channels at the top
- A deeper central cavity

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Material (in the outer runtime) before running the notebook.
2. Set structural parameters in cell 1.2 below
4. Click "Run" > "Run All" to:
   - Create the base slab
   - Apply the cutout pattern
   - Generate the final structure
5. Scroll down to view the resulting structure from multiple angles


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment

### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("", "../../config.yml")

### 1.2. Set up transformation parameters 

In [ ]:
from typing import List

from mat3ra.made.tools.utils.coordinate import CoordinateCondition

MILLER_INDICES = (0, 0, 1)
THICKNESS = 8
VACUUM = 10.0
USE_ORTHOGONAL_Z = True
XY_SUPERCELL_MATRIX = [[10, 0], [0, 10]]


# Custom shape parameters in crystal units of the slab
LEFT_WALL_POSITION = 0.2
RIGHT_WALL_POSITION = 0.8
LEFT_CHANNEL_POSITION = 0.3
RIGHT_CHANNEL_POSITION = 0.7
CHANNEL_DEPTH = 0.8
CAVITY_DEPTH = 0.5


# Custom Coordinate Condition for a transistor mock
class CustomCoordinateCondition(CoordinateCondition):
    """
    Defines a transistor-like structure with side walls, channels, and a central cavity.
    All dimensions are specified as fractions of the total size (0.0 to 1.0).
    """
    # Side walls positioning (x-direction)
    left_wall_position: float = 0.2   # 20% from left edge
    right_wall_position: float = 0.8  # 80% from left edge
    
    # Channel positioning (y-direction)
    left_channel_position: float = 0.3   # 30% from front
    right_channel_position: float = 0.7  # 70% from front
    
    # Height levels (z-direction)
    channel_depth: float = 0.8    # Channels go down to 80% height
    cavity_depth: float = 0.5     # Central cavity goes down to 50% height

    def condition(self, coordinate: List[float]) -> bool:
        """
        Determines which atoms should be removed to create the structure.
        Returns True for positions where atoms should be removed.
        
        The structure consists of:
        1. Side walls: Two vertical walls running along the y-axis
        2. Channels: Two trenches at the top running along the x-axis
        3. Cavity: A deeper central region between the channels
        """
        x, y, z = coordinate  # Unpack coordinates for clarity
        
        # Define the structural elements
        is_in_left_wall = x <= self.left_wall_position
        is_in_right_wall = x >= self.right_wall_position
        
        is_in_left_channel = y <= self.left_channel_position
        is_in_right_channel = y >= self.right_channel_position
        
        # Remove material to create channels at the top
        should_remove_for_channels = (
            z < self.channel_depth and 
            (is_in_left_channel or is_in_right_channel)
        )
        
        # Remove material to create the deeper central cavity
        should_remove_for_cavity = (
            z < self.cavity_depth and                    # Check depth
            self.left_wall_position < x < self.right_wall_position and  # Between walls
            not is_in_left_channel and not is_in_right_channel          # Between channels
        )
        
        return (
            is_in_left_wall or   
            is_in_right_wall or  
            should_remove_for_channels or
            should_remove_for_cavity   
        )


condition = CustomCoordinateCondition().condition

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0]

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=[3, 3, 3], rotation="0x")
visualize(material, repetitions=[3, 3, 3], rotation="-90x")

## 2. Create target material
### 2.1. Create a slab with custom shape cutout

In [ ]:
from mat3ra.made.tools.modify import filter_by_condition_on_coordinates, add_vacuum
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=THICKNESS,
    vacuum=0,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX,
    use_orthogonal_z=True,
)

slab_unit_cell = create_slab(slab_config)
resulting_material = filter_by_condition_on_coordinates(slab_unit_cell, condition)
resulting_material = add_vacuum(resulting_material, VACUUM)

## 3. Visualize the result

In [ ]:
visualize(resulting_material, repetitions=[1, 1, 1], rotation="0x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x")
visualize(resulting_material, repetitions=[1, 1, 1], rotation="-90x,-90y")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
resulting_material.name = "Custom Shape"
set_materials(resulting_material)